In [2]:
import pandas as pd
import requests
import urllib
import json
import time

# 1. IMPORT DATA

### En primer lugar, importamos el archivo que contiene los datos necesarios para entrenar los distintos modelos de Machine Learning que en un futuro utilizaremos.

In [3]:
historic_traffic_data = pd.read_csv(filepath_or_buffer='data/04_2025.csv', sep=';')
historic_traffic_data.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
0,1001,01/04/2025 9:45,C30,2260,8,0,57.0,N,3
1,1001,01/04/2025 10:00,C30,2172,7,0,59.0,N,5
2,1001,01/04/2025 10:15,C30,1680,6,0,54.0,N,2
3,1002,01/04/2025 0:00,C30,0,0,0,0.0,N,5
4,1002,01/04/2025 0:15,C30,0,0,0,0.0,N,5


### A continuación, importamos el .csv que contiene los datos de los diferentes puntos de medida de nivel de tráfico en Madrid. De dicho fichero, hay que extraer dos columnas que las añadiremos al primer DataFrame.

In [4]:
columns = ['distrito', 'id']
measure_points_data = pd.read_csv(filepath_or_buffer='data/pmed_ubicacion_04_2025.csv', sep=';', usecols=columns, encoding='utf-8')
measure_points_data.head()

,distrito,id
0,8.0,6639
1,8.0,3797
2,5.0,6640
3,5.0,6642
4,5.0,6643


### Antes de añadir dichas columnas al DataFrame principal, estudiamos la presencia de valores nulos o en blanco en measure_points_data:

In [5]:
measure_points_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4973 entries, 0 to 4972
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   distrito  4968 non-null   float64
 1   id        4973 non-null   int64  
dtypes: float64(1), int64(1)
memory usage: 77.8 KB


### La columna 'distrito' posee 5 valores nulos, por lo que procederemos a eliminar dichas filas.

In [6]:
measure_points_data.dropna(inplace=True)
measure_points_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4968 entries, 0 to 4972
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   distrito  4968 non-null   float64
 1   id        4968 non-null   int64  
dtypes: float64(1), int64(1)
memory usage: 116.4 KB


### Unimos ambos DataFrames, sabiendo que ambas columnas 'id' de ambos DataFrame representan lo mismo. La adición de la columna 'distrito' al primer DataFrame nos ayudará posteriormente a llevar a cabo agrupaciones o ver en qué distrito se genera una mayor cantidad de atascos. 

In [7]:
data = historic_traffic_data.merge(measure_points_data, left_on='id', right_on='id')
data

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion,distrito
0,1001,01/04/2025 9:45,C30,2260,8,0,57.0,N,3,10.0
1,1001,01/04/2025 10:00,C30,2172,7,0,59.0,N,5,10.0
2,1001,01/04/2025 10:15,C30,1680,6,0,54.0,N,2,10.0
3,1002,01/04/2025 0:00,C30,0,0,0,0.0,N,5,10.0
4,1002,01/04/2025 0:15,C30,0,0,0,0.0,N,5,10.0
...,...,...,...,...,...,...,...,...,...,...
1048570,3805,07/04/2025 12:45,URB,707,2,29,0.0,N,15,13.0
1048571,3805,07/04/2025 13:00,URB,703,3,32,0.0,N,15,13.0
1048572,3805,07/04/2025 13:15,URB,822,4,35,0.0,N,15,13.0
1048573,3805,07/04/2025 13:30,URB,679,3,35,0.0,N,15,13.0


### Al DataFrame principal vamos a añadirle ahora datos sobre las precipitaciones que tuvieron lugar cada día. Para ello, hacemos uso de la API de AEMET.

In [19]:
import os
from dotenv import load_dotenv
from pathlib import Path

def get_data_from_aemet() -> pd.DataFrame:

    load_dotenv()

    main_dataframe = pd.DataFrame()
   
    file_path = Path('data/historic_aemet_data.csv')

    # La franja de tiempo máxima para recolectar datos es de 6 meses. 
    dates = ['2017-01-01T00:00:00UTC', '2017-07-01T00:00:00UTC', '2018-01-01T00:00:00UTC',
             '2018-07-01T00:00:00UTC', '2019-01-01T00:00:00UTC', '2019-07-01T00:00:00UTC', 
             '2020-01-01T00:00:00UTC', '2020-07-01T00:00:00UTC', '2021-01-01T00:00:00UTC', 
             '2021-07-01T00:00:00UTC', '2022-01-01T00:00:00UTC', '2022-07-01T00:00:00UTC', 
             '2023-01-01T00:00:00UTC', '2023-07-01T00:00:00UTC', '2024-01-01T00:00:00UTC', 
             '2024-07-01T00:00:00UTC', '2025-01-01T00:00:00UTC', '2025-04-30T00:00:00UTC']

    API_KEY = os.getenv('AEMET_API_KEY')
    base_url = 'https://opendata.aemet.es/opendata'
    idema = '3195' # Estación

    if not file_path.exists():
        for i in range(len(dates)-1):
            fechaIniStr = dates[i]
            fechaFinStr = dates[i+1]

            endpoint = f"/api/valores/climatologicos/diarios/datos/fechaini/{fechaIniStr}/fechafin/{fechaFinStr}/estacion/{idema}"
            url = base_url + endpoint
            headers = {
                "Accept": "application/json",
                "api_key": API_KEY
            }

            try:
                response = requests.get(url, headers=headers, timeout=15)
                data = response.json()

                if data.get('datos') is not None:
                    weather_data = data['datos']
                    file = urllib.request.urlopen(weather_data)
                    file_content = file.read()
                    weather_data_json = json.loads(file_content)
                    secondary_dataframe = pd.DataFrame(weather_data_json)
                    # Eliminamos duplicados porque las fechas de los extremos se cogen dos veces cada una salvo la primera y la última
                    main_dataframe = pd.concat([main_dataframe, secondary_dataframe], ignore_index=True).drop_duplicates()
                    secondary_dataframe = secondary_dataframe.fillna(0)
                    time.sleep(2)
                else:
                    print(f"No se encontró la clave 'datos' en la respuesta para fechas {fechaIniStr} a {fechaFinStr}")
                    print(f"Error {response.status_code}: {response.text}")
            except Exception as e:
                print(f"Error durante la solicitud para {fechaIniStr} a {fechaFinStr}: {e}")

        return main_dataframe.to_csv('data/historic_aemet_data.csv')
    
    else:
        return pd.read_csv(file_path)


In [20]:
aemet_dataframe = get_data_from_aemet()
aemet_dataframe

### Ambos dataframe tienen 'fecha' como columna común, por lo que ambos dataframe se unirán utilizando dicha columna como clave, pero para poder realizar dicha unión tenemos que llevar a cabo ciertas transformaciones en ambos dataframes.

In [12]:
# DATAFRAME DATA -> Separar la columna fecha en fecha y hora y tipar correctamente ambas columnas resultantes.

data[['fecha', 'hora']] = data['fecha'].str.split(' ', expand=True)
data['fecha'] = pd.to_datetime(data['fecha'], format='%d/%m/%Y').dt.strftime('%d/%m/%Y')
data['hora'] = pd.to_datetime(data['hora'], format='%H:%M').dt.time
data


,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion,distrito,hora
0,1001,01/04/2025,C30,2260,8,0,57.0,N,3,10.0,09:45:00
1,1001,01/04/2025,C30,2172,7,0,59.0,N,5,10.0,10:00:00
2,1001,01/04/2025,C30,1680,6,0,54.0,N,2,10.0,10:15:00
3,1002,01/04/2025,C30,0,0,0,0.0,N,5,10.0,00:00:00
4,1002,01/04/2025,C30,0,0,0,0.0,N,5,10.0,00:15:00
...,...,...,...,...,...,...,...,...,...,...,...
1048570,3805,07/04/2025,URB,707,2,29,0.0,N,15,13.0,12:45:00
1048571,3805,07/04/2025,URB,703,3,32,0.0,N,15,13.0,13:00:00
1048572,3805,07/04/2025,URB,822,4,35,0.0,N,15,13.0,13:15:00
1048573,3805,07/04/2025,URB,679,3,35,0.0,N,15,13.0,13:30:00


In [13]:
# DATAFRAME AEMET -> 
aemet_dataframe['fecha'] = pd.to_datetime(aemet_dataframe['fecha']).dt.strftime('%d/%m/%Y')

In [25]:
required_columns = aemet_dataframe[['fecha', 'prec']]
required_columns

,fecha,prec
0,01/07/2013,"0,0"
1,02/07/2013,"0,0"
2,03/07/2013,"0,0"
3,04/07/2013,"0,0"
4,05/07/2013,"0,0"
...,...,...
4317,26/04/2025,"0,0"
4318,27/04/2025,"0,0"
4319,28/04/2025,"0,0"
4320,29/04/2025,"0,0"


In [27]:
data = data.merge(required_columns, left_on='fecha', right_on='fecha')
data.sort_values(by='fecha', ascending=True, inplace=True)
data

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion,distrito,prec_x,prec_y


In [40]:
from pathlib import Path

# Create a Path object
file_path = Path('my_data/my_file.txt')

# Check if the file exists
if not file_path.exists():
   print("The file exists.")
else:
   print("The file does not exist.")

The file exists.
